In [2]:
# draggable rectangle with the animation blit techniques; see
# http://www.scipy.org/Cookbook/Matplotlib/Animations
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

class DraggableRectangle:
    lock = None  # only one can be animated at a time

    def __init__(self, rect):
        self.rect = rect
        self.press = None
        self.background = None

        self.o_x = None
        self.o_y = None

    def connect(self):
        'connect to all the events we need'
        self.cidpress = self.rect.figure.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.rect.figure.canvas.mpl_connect(
            'button_release_event', self.on_release)
        self.cidmotion = self.rect.figure.canvas.mpl_connect(
            'motion_notify_event', self.on_motion)

    def on_press(self, event):
        'on button press we will see if the mouse is over us and store some data'
        if event.inaxes != self.rect.axes:
            return
        if DraggableRectangle.lock is not None:
            return

        # save the original position for snapping back
        self.o_y = self.rect.get_y()
        self.o_x = self.rect.get_x()

        contains, attrd = self.rect.contains(event)
        if not contains: return
        print('event contains', self.rect.xy)
        x0, y0 = self.rect.xy
        self.press = x0, y0, event.xdata, event.ydata
        DraggableRectangle.lock = self

        # draw everything but the selected rectangle and store the pixel buffer
        canvas = self.rect.figure.canvas
        axes = self.rect.axes
        self.rect.set_animated(True)
        canvas.draw()
        self.background = canvas.copy_from_bbox(self.rect.axes.bbox)

        # now redraw just the rectangle
        axes.draw_artist(self.rect)

        # and blit just the redrawn area
        canvas.blit(axes.bbox)

    def on_motion(self, event):
        'on motion we will move the rect if the mouse is over us'
        if DraggableRectangle.lock is not self:
            return
        if event.inaxes != self.rect.axes:
            return

        x0, y0, xpress, ypress = self.press
        dx = event.xdata - xpress
        dy = event.ydata - ypress
        # self.rect.set_x(x0 + dx)
        self.rect.set_y(y0 + dy)

        canvas = self.rect.figure.canvas
        axes = self.rect.axes
        # restore the background region
        canvas.restore_region(self.background)

        # redraw just the current rectangle
        axes.draw_artist(self.rect)

        # blit just the redrawn area
        canvas.blit(axes.bbox)

    def on_release(self, event):
        'on release we reset the press data'
        if DraggableRectangle.lock is not self:
            return

        # check if it is released past half of the above or below
        # get y of above and below
        if self.i == 0:
            y_below = None
        else:
            y_below = drs[self.i - 1].rect.get_y()

        if self.i == len(drs) - 1:
            y_above = None
        else:
            y_above = drs[self.i + 1].rect.get_y()

        # if the released rect is not above
        if y_above is None or y_below is None:
            self.rect.set_y(self.o_y)

        if self.rect.get_y() <= (y_above + self.rect.get_height() / 2):
            self.rect.set_y(self.o_y)
        elif self.rect.get_y() >= (y_below + self.rect.get_height() / 2):
            self.rect.set_y(self.o_y)

        self.press = None
        DraggableRectangle.lock = None

        # turn off the rect animation property and reset the background
        self.rect.set_animated(False)
        self.background = None

        # redraw the full figure
        self.rect.figure.canvas.draw()

    def disconnect(self):
        'disconnect all the stored connection ids'
        self.rect.figure.canvas.mpl_disconnect(self.cidpress)
        self.rect.figure.canvas.mpl_disconnect(self.cidrelease)
        self.rect.figure.canvas.mpl_disconnect(self.cidmotion)


fig = plt.figure()
ax = fig.add_subplot(111)
rects = ax.barh(range(10), 20 * np.random.rand(10))
# list of DraggableRectangle instances
drs = []

for i, rect in enumerate(rects):
    # create draggable rectangle from all created bars
    dr = DraggableRectangle(rect)
    dr.i = i
    dr.connect()
    drs.append(dr)  # add instance to list of all rectangles
print(drs)
plt.show()


[<__main__.DraggableRectangle object at 0x7f08121b2b00>, <__main__.DraggableRectangle object at 0x7f081216e400>, <__main__.DraggableRectangle object at 0x7f081216e9e8>, <__main__.DraggableRectangle object at 0x7f081216ed68>, <__main__.DraggableRectangle object at 0x7f081217e320>, <__main__.DraggableRectangle object at 0x7f081217e198>, <__main__.DraggableRectangle object at 0x7f0808098c50>, <__main__.DraggableRectangle object at 0x7f080804f860>, <__main__.DraggableRectangle object at 0x7f080804f898>, <__main__.DraggableRectangle object at 0x7f080804fb00>]
event contains (0.0, 8.6)


In [4]:
rect.get_x()

0.0

In [5]:
rect.get_y()

8.6

In [9]:
rect.get_height()

0.8

In [12]:
a = drs[0]

In [14]:
a.rect.get_x()

0.0

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5
class DraggableRectangle:
    lock = None  # only one can be animated at a time

    def __init__(self, rect, parent=None):
        self.rect = rect
        self.parent = parent
        self.press = None
        self.background = None

        self.o_x = None
        self.o_y = None

    def connect(self):
        'connect to all the events we need'
        self.cidpress = self.rect.figure.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.rect.figure.canvas.mpl_connect(
            'button_release_event', self.on_release)
        self.cidmotion = self.rect.figure.canvas.mpl_connect(
            'motion_notify_event', self.on_motion)

    def on_press(self, event):
        'on button press we will see if the mouse is over us and store some data'
        if event.inaxes != self.rect.axes:
            return
        if DraggableRectangle.lock is not None:
            return

        # save the original position for snapping back
        self.o_y = self.rect.get_y()
        self.o_x = self.rect.get_x()

        contains, attrd = self.rect.contains(event)
        if not contains: return
        print('event contains', self.rect.xy)
        x0, y0 = self.rect.xy
        self.press = x0, y0, event.xdata, event.ydata
        DraggableRectangle.lock = self

        # draw everything but the selected rectangle and store the pixel buffer
        canvas = self.rect.figure.canvas
        axes = self.rect.axes
        self.rect.set_animated(True)
        canvas.draw()
        self.background = canvas.copy_from_bbox(self.rect.axes.bbox)

        # now redraw just the rectangle
        axes.draw_artist(self.rect)

        # and blit just the redrawn area
        canvas.blit(axes.bbox)

    def on_motion(self, event):
        'on motion we will move the rect if the mouse is over us'
        if DraggableRectangle.lock is not self:
            return
        if event.inaxes != self.rect.axes:
            return

        x0, y0, xpress, ypress = self.press
        dx = event.xdata - xpress
        dy = event.ydata - ypress
        # self.rect.set_x(x0 + dx)
        self.rect.set_y(y0 + dy)

        if self.rect.get_y() > self.parent.bars[self.i + 1].rect.get_y() + self.rect.get_height() / 2:
            # reduce i for above
            self.parent.bars[self.i + 1].i -= 1
            y_bot = self.get_y()
            self.rect.set_y(self.parent.bars[self.i + 1].rect.get_y())
            self.parent.bars[self.i + 1].set_y(y_bot)
            # swap positions
            self.parent.bars[self.i], self.parent.bars[self.i + 1] = self.parent.bars[self.i + 1], self.parent.bars[self.i]
            # increase i for self
            self.i += 1


        canvas = self.rect.figure.canvas
        axes = self.rect.axes
        # restore the background region
        canvas.restore_region(self.background)

        # redraw just the current rectangle
        axes.draw_artist(self.rect)

        # blit just the redrawn area
        canvas.blit(axes.bbox)

    def on_release(self, event):
        'on release we reset the press data'
        if DraggableRectangle.lock is not self:
            return

        # check if it is released past half of the above or below
        # get y of above and below
        if self.i == 0:
            y_below = None
        else:
            y_below = self.parent.bars[self.i - 1].rect.get_y()

        if self.i == len(self.parent.bars) - 1:
            y_above = None
        else:
            y_above = self.parent.bars[self.i + 1].rect.get_y()

        # if the released rect is not above
        if y_above is None or y_below is None:
            self.rect.set_y(self.o_y)

        if self.rect.get_y() <= (y_above + self.rect.get_height() / 2):
            self.rect.set_y(self.o_y)
        elif self.rect.get_y() >= (y_below + self.rect.get_height() / 2):
            self.rect.set_y(self.o_y)

        self.press = None
        DraggableRectangle.lock = None

        # turn off the rect animation property and reset the background
        self.rect.set_animated(False)
        self.background = None

        # redraw the full figure
        self.rect.figure.canvas.draw()

    def disconnect(self):
        'disconnect all the stored connection ids'
        self.rect.figure.canvas.mpl_disconnect(self.cidpress)
        self.rect.figure.canvas.mpl_disconnect(self.cidrelease)
        self.rect.figure.canvas.mpl_disconnect(self.cidmotion)


In [ ]:
import numpy as np
class DaPlot:
    def __init__(self, formations, series):
        self.formations = formations
        self.series = series

        self.fig = plt.figure()
        self.ax = self.fig.add_subplot(111)

        self.barplot = self.ax.barh(range(len(self.formations)), [1 for i in range(len(self.formations))])
        self.bars = []

        self.ax.xaxis.set_visible(False)
        self.ax.set_yticklabels(self.formations)

        for i, entry in enumerate(self.barplot):
            bar = DraggableRectangle(entry, parent=self)
            bar.i = i
            bar.connect()
            self.bars.append(bar)

        plt.show()

    #fig = plt.figure()
    #ax = fig.add_subplot(111)
    #formations = ax.barh(range(10), 20 * np.random.rand(10))
    # list of DraggableRectangle instances
    #drs = []

    #for i, fmt in enumerate(formations):
    #    # create draggable rectangle from all created bars
    #    dr = DraggableRectangle(fmt)
    #    dr.i = i
    #    dr.connect()
    #    drs.append(dr)  # add instance to list of all rectangles
    #print(drs)
    #plt.show()

import pandas as pn

formations = np.array(['Layer 2', 'Layer 3', 'Layer 4', 'Layer 5', 'Layer 6', 'Fault'], dtype=object)
series = pn.Index(['fault', 'Rest'], dtype='object')

plot = DaPlot(formations, series)

In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5
class DraggableRectangle:
    def __init__(self, rect, f):
        self.rect = rect
        self.f = f
        self.press = None

    def connect(self):
        'connect to all the events we need'
        self.cidpress = self.rect.figure.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.rect.figure.canvas.mpl_connect(
            'button_release_event', self.on_release)
        self.cidmotion = self.rect.figure.canvas.mpl_connect(
            'motion_notify_event', self.on_motion)
        self.figure.canvas.mpl_connect('axes_enter_event', enter_axes)

    def on_press(self, event):
        'on button press we will see if the mouse is over us and store some data'
        if event.inaxes != self.rect.axes: return

        contains, attrd = self.rect.contains(event)
        if not contains: return
        print('event contains', self.rect.xy)
        x0, y0 = self.rect.xy
        self.press = x0, y0, event.xdata, event.ydata

    def on_motion(self, event):
        'on motion we will move the rect if the mouse is over us'
        if self.press is None: return
        if event.inaxes != self.rect.axes: return
        x0, y0, xpress, ypress = self.press
        dx = event.xdata - xpress
        dy = event.ydata - ypress
        #print('x0=%f, xpress=%f, event.xdata=%f, dx=%f, x0+dx=%f' %
        #      (x0, xpress, event.xdata, dx, x0+dx))
        self.rect.set_x(x0+dx)
        self.rect.set_y(y0+dy)

        self.rect.figure.canvas.draw()


    def on_release(self, event, anchor = 2):
        'on release we reset the press data'
        self.press = None
        self.rect.figure.canvas.draw()
        self.rect.set_y(anchor)
        
        self.rect.set_animated(False)
        self.background = None

        # redraw the full figure
        self.rect.figure.canvas.draw()
        return 'things'

    def disconnect(self):
        'disconnect all the stored connection ids'
        self.rect.figure.canvas.mpl_disconnect(self.cidpress)
        self.rect.figure.canvas.mpl_disconnect(self.cidrelease)
        self.rect.figure.canvas.mpl_disconnect(self.cidmotion)

fig = plt.figure()
ax = fig.add_subplot(111)
rects = ax.barh(1, 10, 5)
rects = ax.barh(range(10), 20*np.random.rand(10))


drs = []
for e, rect in enumerate(rects):
    dr = DraggableRectangle(rect, e)
    dr.connect()
    drs.append(dr)

plt.show()

AttributeError: 'DraggableRectangle' object has no attribute 'figure'

In [8]:
ax

event contains (0.0, 2)


In [3]:
import matplotlib.pyplot as plt
%matplotlib qt5

def enter_axes(event):
    print('enter_axes', event.inaxes)
    event.inaxes.patch.set_facecolor('yellow')
    event.canvas.draw()

def leave_axes(event):
    print('leave_axes', event.inaxes)
    event.inaxes.patch.set_facecolor('white')
    event.canvas.draw()

def enter_figure(event):
    print('enter_figure', event.canvas.figure)
    event.canvas.figure.patch.set_facecolor('red')
    event.canvas.draw()

def leave_figure(event):
    print('leave_figure', event.canvas.figure)
    event.canvas.figure.patch.set_facecolor('grey')
    event.canvas.draw()

fig1 = plt.figure()
fig1.suptitle('mouse hover over figure or axes to trigger events')
ax1 = fig1.add_subplot(211)
ax2 = fig1.add_subplot(212)

fig1.canvas.mpl_connect('figure_enter_event', enter_figure)
fig1.canvas.mpl_connect('figure_leave_event', leave_figure)
fig1.canvas.mpl_connect('axes_enter_event', enter_axes)
fig1.canvas.mpl_connect('axes_leave_event', leave_axes)

fig2 = plt.figure()
fig2.suptitle('mouse hover over figure or axes to trigger events')
ax1 = fig2.add_subplot(211)
ax2 = fig2.add_subplot(212)

fig2.canvas.mpl_connect('figure_enter_event', enter_figure)
fig2.canvas.mpl_connect('figure_leave_event', leave_figure)
fig2.canvas.mpl_connect('axes_enter_event', enter_axes)
fig2.canvas.mpl_connect('axes_leave_event', leave_axes)

plt.show()

enter_figure Figure(640x480)
enter_axes Axes(0.125,0.11;0.775x0.35)
leave_axes Axes(0.125,0.11;0.775x0.35)
enter_axes Axes(0.125,0.53;0.775x0.35)
leave_axes Axes(0.125,0.53;0.775x0.35)
leave_figure Figure(640x480)
enter_figure Figure(640x480)
leave_figure Figure(640x480)


In [6]:
ax.barh(range(10), 20*np.random.rand(10))[:]

(<matplotlib.patches.Rectangle at 0x7fb2cf572a58>,
 <matplotlib.patches.Rectangle at 0x7fb2cf5a34a8>)

In [15]:
ax.barh?

In [10]:
np.linspace(0, 10, 4, endpoint=False)

array([ 0. ,  2.5,  5. ,  7.5])

In [8]:
np.linspace?

In [ ]:
ax.set_ylim